| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`Select` |Transforms each input element with the given lambda expression |SELECT
|`SelectMany` |Transforms each input element, and then flattens and concatenates<br>the resultant subsequences |INNER JOIN,<br>LEFT OUTER JOIN,<br>CROSS JOIN

### Select

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Result selector |TSource => TResult or (TSource,int) => TResult

In [ ]:
public static IEnumerable<TResult> Select<TSource,TResult>
(this IEnumerable<TSource> source, Func<TSource,TResult> selector)
{
    foreach (TSource element in source)
        yield return selector (element);
}

With ***Select***, you always get the `same number of elements` that you started with. Each element, however, can be `transformed` in any manner by the lambda function.

In [1]:
using System.Drawing;

//The following selects the names of all fonts installed on the computer
IEnumerable<string> query = 
    from f in FontFamily.Families
    select f.Name;

IEnumerable<string> lamdaQuery = FontFamily.Families.Select (f => f.Name);

Error: (5,15): error CS0103: The name 'FontFamily' does not exist in the current context
(13,34): error CS0103: The name 'FontFamily' does not exist in the current context

In [ ]:
using System.Drawing;
//Select statements are often used to project into anonymous types:
var query =
from f in FontFamily.Families
select new { f.Name, LineSpacing = f.GetLineSpacing (FontStyle.Bold) };

***Indexed projection***

In [3]:
string[] names = { "Tom", "Dick", "Harry", "Mary", "Jay" };
IEnumerable<string> query = names
.Select ((s,i) => i + "=" + s); // { "0=Tom", "1=Dick", ... }

foreach(var item in query)
    Console.WriteLine( item);

0=Tom
1=Dick
2=Harry
3=Mary
4=Jay


***Select subqueries and object hierarchies***

In [ ]:
using System.IO;

string tempPath = Path.GetTempPath();
DirectoryInfo[] dirs = new DirectoryInfo (tempPath).GetDirectories();

var query =
from d in dirs
where (d.Attributes & FileAttributes.System) == 0
select new
{
    DirectoryName = d.FullName,
    Created = d.CreationTime,
    Files = from f in d.GetFiles()
        where (f.Attributes & FileAttributes.Hidden) == 0
        select new { FileName = f.Name, f.Length, }
};

foreach (var dirFiles in query)
{
    Console.WriteLine ("Directory: " + dirFiles.DirectoryName);
    foreach (var file in dirFiles.Files)
    Console.WriteLine (" " + file.FileName + " Len: " + file.Length);
}


***correlated subquery***  
subquery is **correlated** if it `references` an object in the `outer query`

With ***local queries***, a `subquery` `within a Select` causes **double-deferred execution**.

<div dir="rtl" style="width:80%; margin:auto">
هر پوشه تنها زمانی مورد بررسی قرار می‌گیرد که در حلقه بیرونی به آن رسیده شود و فایل‌های هر پوشه نیز فقط زمانی بررسی و فیلتر می‌شوند که به آن‌ها در حلقه داخلی نیاز باشد. این رویکرد منجر به صرفه‌جویی در منابع و افزایش کارایی می‌شود، چرا که تنها داده‌های مورد نیاز در هر لحظه پردازش می‌شوند.

</div>

***Subqueries and joins in EF Core***

In [ ]:
var query =
    from c in dbContext.Customers
    select new {
            c.Name,
            Purchases = (from p in dbContext.Purchases
            where p.CustomerID == c.ID && p.Price > 100
            select new { p.Description, p.Price })
        .ToList()
};

foreach (var namePurchases in query)
{
    Console.WriteLine ("Customer: " + namePurchases.Name);
    foreach (var purchaseDetail in namePurchases.Purchases)
        Console.WriteLine (" - $$$: " + purchaseDetail.Price);
}

<div dir="rtl" style="width:80%; margin:auto">
<h3>جوین در دیتابیس‌های رابطه‌ای</h3>
<p>در یک پایگاه داده رابطه‌ای معمولی، وقتی از عملیات جوین (Join) استفاده می‌کنیم، دو جدول مختلف را بر اساس یک یا چند شرط مشترک به هم متصل می‌کنیم. نتیجه یک جدول دو بعدی است که در آن هر سطر ممکن است داده‌ها از هر دو جدول را در خود داشته باشد. این جدول نتیجه معمولاً به صورت یک نما (view) یا یک مجموعه داده‌های فلت (flat dataset) است که همه اطلاعات مربوطه در سطرهای متوالی قرار دارند.</p>

<h3>پرس و جو در Entity Framework Core</h3>
<p>در مقابل، Entity Framework Core با استفاده از LINQ این امکان را فراهم می‌کند که به جای ساخت یک جدول دو بعدی فلت، داده‌ها را به صورت سلسله مراتبی سازمان‌دهی کنیم. </p>

<h3>مزایای ساختار سلسله مراتبی</h3>

<p>این رویکرد ساختارمند بودن داده‌ها به توسعه‌دهندگان کمک می‌کند تا به طور طبیعی با داده‌ها به شکلی که بیشتر منعکس‌کننده ساختار شیء‌گرای برنامه است، کار کنند. برخلاف نمایش دو بعدی و فلت، این ساختار به توسعه‌دهندگان اجازه می‌دهد که روابط پیچیده و سلسله مراتبی میان انواع مختلف داده‌ها را به طور مؤثرتری مدیریت کنند.</p>
</div>

In [ ]:
// dont need to use p.CustomerID == c.ID
var query =
    from c in dbContext.Customers
    select new {
            c.Name,
            Purchases = 
            (from p in c.Purchases
            where  p.Price > 1000
            select new { p.Description, p.Price })
        .ToList()
};

In [ ]:
var query =
    from c in dbContext.Customers
    where c.Purchases.Any (p => p.Price > 1000)
    select new {
            c.Name,
            Purchases = 
            (from p in c.Purchases
            where p.Price > 1000
            select new { p.Description, p.Price })
        .ToList()
};

In [ ]:
//we can do above code by below code 
var query =
    from c in dbContext.Customers
    let highValueP = from p in c.Purchases
                    where p.Price > 1000
                    select new { p.Description, p.Price }
    where highValueP.Any()
    select new { c.Name, Purchases = highValueP };

***Projecting into concrete types***

### SelectMany

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Result selector |TSource => IEnumerable<TResult> <br> (TSource,int) => IEnumerable<TResult>  (Prohibited with EF Core)

#### Query syntax
    from identifier1 in enumerable-expression1
    from identifier2 in enumerable-expression2



In [ ]:
//Enumerable implementation

public static IEnumerable<TResult> SelectMany<TSource,TResult>
(IEnumerable<TSource> source,
Func <TSource,IEnumerable<TResult>> selector)
{
    foreach (TSource element in source)
        foreach (TResult subElement in selector (element))
            yield return subElement;
}

In [ ]:
string[] fullNames = { "Anne Williams", "John Fred Smith", "Sue Green" };

//we want create list like This
//"Anne", "Williams", "John", "Fred", "Smith", "Sue", Green"

IEnumerable<string> query = fullNames.SelectMany (name => name.Split());

`SelectMany` is supported in ***query syntax*** and is invoked by having an `additional generator`  
in other words, an extra `from` clause in the query.   
  
The `from` ***keyword*** has **two meanings** in *query syntax*.   
  
- ***At the start*** of a query, it introduces the **original** `range variable` and input sequence
- ***Anywhere else*** in the query, it translates to `SelectMany`

In [1]:
string[] fullNames = { "Anne Williams", "John Fred Smith", "Sue Green" };

//SelectMany  in query syntax
IEnumerable<string> query =
    from fullName in fullNames
    from name in fullName.Split() // Translates to SelectMany
    select name;

foreach(var item in query)
    Console.WriteLine(item);

Anne
Williams
John
Fred
Smith
Sue
Green


#### Multiple range variables 

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
  <p>در <code>LINQ query syntax</code>، متغیرهای دامنه (مانند <code>name</code> و <code>fullName</code> در این مثال) تا پایان کوئری یا تا زمانی که به یک عبارت <code>into</code> برسند، در دسترس می‌مانند. این مزیت در کوئری‌های پیچیده‌تر که نیاز به دسترسی به چندین متغیر دامنه دارند، کاربردی است. به‌طور مثال، در این کوئری:</p>
    
</div>

In [ ]:
string[] fullNames = { "Anne Williams", "John Fred Smith", "Sue Green" };

//SelectMany  in query syntax
IEnumerable<string> query =
    from fullName in fullNames
    from name in fullName.Split() // Translates to SelectMany
    order by
    select name;

foreach(var item in query)
    Console.WriteLine(item);

Anne
Williams
John
Fred
Smith
Sue
Green


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>در <code>fluent syntax</code>، کار کمی پیچیده‌تر می‌شود، چون در هنگام استفاده از <code>SelectMany</code> به‌صورت مستقیم، عنصر بیرونی (یعنی <code>fullName</code>) از بین می‌رود و دیگر قابل دسترسی نیست. به‌عبارتی دیگر، <code>SelectMany</code> تنها فرزندان (در اینجا کلمات جدا شده توسط <code>Split</code>) را به‌صورت تخت (flat) باز می‌گرداند.</p>
<p>برای حل این مشکل، می‌توانیم عنصر بیرونی را با هر عنصر داخلی حمل کنیم و آن را در یک نوع ناشناس ذخیره کنیم، به‌گونه‌ای که هر دو عنصر <code>name</code> و <code>fullName</code> در یک ساختار موقتی ذخیره شوند. این کار به این شکل انجام می‌شود:</p>
</div>

In [ ]:
string[] fullNames = { "Anne Williams", "Sue Green", "John Fred Smith" };

IEnumerable<string> query = fullNames
    .SelectMany(fName => fName.Split().Select(name => new { name, fName }))
    .OrderBy(x => x.fName)
    .ThenBy(x => x.name)
    .Select(x => x.name + " came from " + x.fName);


foreach(var item in query)
    Console.WriteLine(item);

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p> دو الگوی اصلی در استفاده از syntax query در LINQ  برای کار با چندین متغیر دامنه وجود دارد. این الگوها شامل گسترش و تخت کردن زیرترتیب‌ها (subsequences) و انجام ضرب دکارتی (Cartesian Product) یا به عبارت دیگر cross join است.</p>

<h3>1. گسترش و تخت کردن زیرترتیب‌ها</h3>

<p>در این الگو، شما یک ویژگی یا متد را در یک متغیر دامنه (range variable) فراخوانی می‌کنید تا به زیرترتیب‌ها دسترسی پیدا کنید. در اینجا یک نمونه از این الگو آورده شده:</p>
</div>

In [ ]:
//expanding and flattening subsequences

string[] fullNames = { "Anne Williams", "Sue Green", "John Fred Smith" };

IEnumerable<string> query =
    from fullName in fullNames
    from name in fullName.Split()
    select name + " came from " + fullName;

//in Ef queries 


//Here, we’ve expanded each customer into a subsequence of purchases.

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>در این کوئری، از <code>fullNames</code> به <code>name</code> تغییر می‌دهیم. به عبارت دیگر، از لیستی از نام‌های کامل (مانند "John Doe") به کلمات جداگانه (مانند "John" و "Doe") گسترش می‌دهیم. این کار با <code>Split</code> انجام می‌شود که نام کامل را به کلمات تقسیم می‌کند. به این ترتیب، از یک ترتیب کلی به زیرترتیب‌ها دسترسی پیدا کرده‌ایم.</p>

<p>مثال مشابهی در Entity Framework (EF Core) وجود دارد که شما به لیست مجموعه‌های ناوبری دسترسی پیدا می‌کنید. در مثال زیر، تمامی مشتری‌ها به همراه خریدهای آنها لیست می‌شوند:</p>
</div>

In [ ]:
IEnumerable<string> query1 = 
    from c in dbContext.Customers
    from p in c.Purchases
    select c.Name + " bought a " + p.Description;

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>2. ضرب دکارتی (Cartesian Product) یا cross join</h3>
<p>دومین الگو زمانی استفاده می‌شود که هر عنصر از یک ترتیب را با هر عنصر از ترتیب دیگری جفت می‌کنید. این نوع کوئری به عنوان ضرب دکارتی شناخته می‌شود و در آن ترتیب‌های انتخاب شده به هم مرتبط نیستند.</p>
</div>

In [ ]:
//cartesian product, or cross join

int[] numbers = { 1, 2, 3 }; string[] letters = { "a", "b" };
IEnumerable<string> query = 
from n in numbers
from l in letters
select n.ToString() + l;
// RESULT: { "1a", "1b", "2a", "2b", "3a", "3b" }


### Joining with SelectMany

In [ ]:
//You can use SelectMany to join two sequences simply by filtering the results of a
//cross product.

string[] players = { "Tom", "Jay", "Mary" };

IEnumerable<string> query = from name1 in players
                            from name2 in players
                            select name1 + " vs " + name2;
                            
//RESULT: { "Tom vs Tom", "Tom vs Jay", "Tom vs Mary",
// "Jay vs Tom", "Jay vs Jay", "Jay vs Mary",
// "Mary vs Tom", "Mary vs "Jay", "Mary vs Mary" }

In [ ]:
string[] players = { "Tom", "Jay", "Mary" };

IEnumerable<string> query = from name1 in players
                            from name2 in players
                            where name1.CompareTo (name2) != 0
                            orderby name1, name2
                            select name1 + " vs " + name2;
                            
//RESULT: { "Jay vs Mary", "Jay vs Tom", "Mary vs Tom" }


### SelectMany in EF Core

***SelectMany*** in **EF Core** can perform `cross joins`,` non-equi joins`, `inner joins`, and `left outer joins`.

In [ ]:
--non-equi join

SELECT 
    Employees.Name, 
    Departments.DepartmentName 
FROM  Employees
JOIN  Departments  ON  Employees.Age > Departments.MinRequiredAge

-- جوین غیر مساوی ،  استفاده از عملگرهای بزرگتر (>), کوچکتر (<), 
-- بزرگتر یا مساوی (>=)، یا کوچکتر یا مساوی (<=) در شروط جوین می‌تواند

In [ ]:
//cross join

var query = from c in dbContext.Customers
            from p in dbContext.Purchases
            select c.Name + " might have bought a " + p.Description;


var query1 = dbContext.Customers
    .SelectMany(c => dbContext.Purchases, (c, p) => c.Name + " might have bought a " + p.Description);


//matches every customer to every purchase

In [ ]:
//equi-join (base on equel "=" join) or inner join

 var query = from c in dbContext.Customers
              from p in dbContext.Purchases.Where(p => p.CustomerID == c.ID)
              select c.Name + " bought a " + p.Description;
            
var query1 = dbContext.Customers
    .SelectMany(c => dbContext.Purchases
        .Where(p => p.CustomerID == c.ID), (c, p) => new
    {
        CustomerName = c.Name,
        PurchaseDescription = p.Description
    });


In [ ]:
//left join

 var query = from c in dbContext.Customers
              from p in dbContext.Purchases.Where(p => p.CustomerID == c.ID).DefaultIfEmpty()
              select c.Name + " bought a " + p.Description;

var query1 = dbContext.Customers
    .SelectMany(c => dbContext.Purchases
        .Where(p => p.CustomerID == c.ID)
        .DefaultIfEmpty(), (c, p) => new
    {
        CustomerName = c.Name,
        PurchaseDescription = p != null ? p.Description : "No Purchase",
        PurchasePrice = p?.Price
    });


In [ ]:
//none - equi-join
var query = from c in dbContext.Customers
            from p in dbContext.Purchases
            where p.CustomerID == c.ID && c.ID < p.Price
            select c.Name + " bought a " + p.Description;

var query1 = dbContext.Customers
    .SelectMany(c => dbContext.Purchases
        .Where(p => p.CustomerID == c.ID && p.Price > 100), (c, p) => new
    {
        CustomerName = c.Name,
        PurchaseDescription = p.Description,
        PurchasePrice = p.Price
    });

   

In [ ]:
//If you have collection navigation properties in your entities,
//you dont need where

var query = from c in dbContext.Customers
            from p in c.Purchases
            select c.Name + " bought a " + p.Description;

In [ ]:
//You can add where clauses to such a query for additional filtering.
var query = 
    from c in dbContext.Customers
    where c.Name.StartsWith ("T")
    from p in c.Purchases
    select new { c.Name, p.Description };

// in EF above code equel to below code

var query1 = 
    from c in dbContext.Customers
    from p in c.Purchases
    where c.Name.StartsWith ("T")
    select new { c.Name, p.Description };



for ***local query***, moving the `where` clause `down` would make it `less efficient`. With ***local queries***, you should `filter` **before** `joining`.

In [ ]:
//You can introduce new tables into the mix with additional from clauses.

var query = 
from c in dbContext.Customers
from p in c.Purchases
from pi in p.PurchaseItems
select new { c.Name, p.Description, pi.Detail };

In [ ]:
//To include data from a parent table
//(via a navigation property), you don’t add a from clause

var query = 
from c in dbContext.Customers
select new { Name = c.Name, SalesPerson = c.SalesPerson.Name };

//You don’t use SelectMany in this case because there’s no subcollection to flatten.

### Outer joins with SelectMany

<div dir="rtl" style="width:90%;margin:auto;">
<h2>انواع join ها</h2>
<h3>INNER JOIN</h3>
<ul><li><strong>INNER JOIN</strong>: اتصال داده‌ها بر اساس شرط مشترک بین دو جدول؛ تنها ردیف‌هایی که در هر دو جدول مطابقت دارند، نمایش داده می‌شوند.</li></ul>
<h3>OUTER JOIN</h3>
<ul>
<li><strong>LEFT (OUTER) JOIN</strong>: نمایش تمامی ردیف‌های جدول سمت چپ و ردیف‌های مطابق از جدول سمت راست. ردیف‌هایی از جدول سمت چپ که مطابقت ندارند، با مقادیر <code>NULL</code> برای ستون‌های جدول سمت راست نمایش داده می‌شوند.</li>
<li><strong>RIGHT (OUTER) JOIN</strong>: نمایش تمامی ردیف‌های جدول سمت راست و ردیف‌های مطابق از جدول سمت چپ. ردیف‌هایی از جدول سمت راست که مطابقت ندارند، با مقادیر <code>NULL</code> برای ستون‌های جدول سمت چپ نمایش داده می‌شوند.</li>
<li><strong>FULL (OUTER) JOIN</strong>: ترکیبی از <code>LEFT JOIN</code> و <code>RIGHT JOIN</code> که تمامی ردیف‌ها از هر دو جدول را نمایش می‌دهد. اگر مطابقتی در یکی از جداول وجود نداشته باشد، مقادیر <code>NULL</code> برای ستون‌های جدول مقابل ظاهر می‌شود.</li></ul>

<h3>CROSS JOIN</h3>
<ul><li><strong>CROSS JOIN</strong>: این جوین بدون نیاز به شرط اتصال، هر ردیف از جدول اول را با هر ردیف از جدول دوم ترکیب می‌کند. نتیجه، جدولی است که شامل ترکیبات ممکن هر دو جدول است.</li></ul>

<h3>SELF JOIN</h3>
<ul><li><strong>SELF JOIN</strong>: این نوع جوین برای اتصال یک جدول به خودش به کار می‌رود، معمولاً برای مقایسه ردیف‌ها درون یک جدول با یکدیگر استفاده می‌شود.</li></ul>

<h3>NATURAL JOIN</h3>
<ul><li><strong>NATURAL JOIN</strong>: این نوع اتصال داده‌ها را بر اساس ستون‌های مشترک با نام‌های یکسان در دو جدول ادغام می‌کند، بدون اینکه نیاز به مشخص کردن شرط مطابقت باشد.</li></ul>
</div>